In [3]:
import math
import pandas as pd
import numpy as np

def f(X):
    #the function given
    return ((5*X[0]-X[1])**4 + (X[0]-2)**2 + X[0] - 2*X[1] + 12)

In [4]:
def squared_euclidean_dist(x,y):
    diff = x - y
    dist = diff @ diff
    return(dist)

In [13]:
def simplex_method(f, X = pd.DataFrame({'x':[2,1,0], 'y' : [1,2,0]}), r = 1, e = 4, c = 0.25, epsilon=1e-3):
    # f is the function to apply the simplex method
    # dfx is the partival derivative with respect to x
    # dfy is the partial derivative with respect to y
    # X is the starting point set for the newton method, (1,0), (0,1), (1,1) if not given
    # epsilon is used to know when to stop, it's value can determine the precision
    
    # r = reflection coefficient
    # e = expansion coefficient
    # c = contraction coefficient

    # xh = x for highest obj value within starting points
    # xl = x for lowest obj value within starting points
    
    #create empty lists to report
    list_f = [] ## new f(x) values at the end of nth iteration
    list_xmean = [] ## mean x values (excluding xh)
    list_xh = [] ## xh values
    list_xl = [] ## xl values
    list_xnew = [] ## new x values at the end of nth iteration
    type_ = [] ## type of the creation

    i = 0 ## iteration number
    
    
    while(True):
        h_indx = np.argmax([f(row) for index, row in X.iterrows()], axis=0) ## index for argmax x
        l_indx = np.argmin([f(row) for index, row in X.iterrows()], axis=0) ## index for argmin x 
        
        xh = X.iloc[h_indx, :] ## point with highest obj value within set
        xl = X.iloc[l_indx, :] ## point with lowest obj value within set
        
        x_mean = X.loc[(X.x != xh[0]) | (X.y != xh[1]),:].mean()
        
        list_xh.append(f'({xh[0]},{xh[1]})')
        list_xl.append(f'({xl[0]},{xl[1]})')
        list_xmean.append(f'({x_mean[0]},{x_mean[1]})')
        
        ## *reflection*
        xr = x_mean + r*(x_mean - xh) ## r is the reflection coefficient
        
        if f(xl) > f(xr) :
            ## *expansion*
            xe = x_mean + e*(xr - x_mean) ## e is the expansion coefficient
            if f(xr) > f(xe):
                X.iloc[h_indx, :] = xe ## expansion is succeeded, new simplex is obtained
                type_.append('E')
                list_xnew.append([xe])
                list_f.append([f(xe)])
             
            else:
                X.iloc[h_indx, :] = xr ## expansion is failed, xh and xr are replaced to obtain a new simplex
                type_.append('R')
                list_xnew.append([xr])
                list_f.append([f(xr)])
        else:
            if(f(X.iloc[np.argmax([f(row) for index, row in  X.iloc[X.index != h_indx,:].iterrows()], axis=0),:]) > f(xr)):
                X.iloc[h_indx, :] = xr ## xh and xr are replaced to obtain a new simplex
                type_.append('R')
                list_xnew.append([xr])
                list_f.append([f(xr)])
                
            else:
                ## *contraction*
                if f(xh) > f(xr):
                    xh_new = xr 
                else:
                    xh_new = xh
                xc = x_mean + c*(xh_new - x_mean) ## c is the contraction coefficient
                if(f(xc) <= f(xh_new)):
                    X.iloc[h_indx, :] = xc
                    type_.append('C')
                    list_xnew.append([xc])
                    list_f.append([f(xc)])
                else:
                    for index, row in X.iterrows():
                        row[0] = row[0] + (1/2)*(xl[0]-row[0])
                        row[1] = row[1] + (1/2)*(xl[1]-row[0])
                    type_.append('C')
                    list_xnew.append([(row[0],row[1]) for index, row in X.iterrows()])
                    list_f.append([f(row) for index,row in X.iterrows()])
        
        i += 1
        if((sum([squared_euclidean_dist(X.iloc[i,:], x_mean) for i in range(X.shape[0])])/(X.shape[0]))**(1/2) < epsilon):
            break
    return pd.DataFrame({'X_bar':list_xmean, 'X_h':list_xh, 
                         "X_l": list_xl, 'X_new':list_xnew, 'f(X_new)':list_f,
                         "type":type_}).rename_axis('Iteration')

In [14]:
simplex_method(f)

,X_bar,X_h,X_l,X_new,f(X_new),type
Iteration,,,,,,
0,"(0.5,1.0)","(2,1)","(0,0)","[[-1.0, 1.0]]",[1314.0],R
1,"(0.5,1.0)","(-1,1)","(0,0)","[[0.125, 1.0]]",[13.660400390625],C
2,"(0.0625,0.5)","(1.0,2.0)","(0.125,1.0)","[[0.296875, 0.875]]",[13.585401594638824],C
3,"(0.2109375,0.9375)","(0.0,0.0)","(0.296875,0.875)","[[1.0546875, 4.6875]]",[4.6911737360060215],E
4,"(0.67578125,2.78125)","(0.125,1.0)","(1.0546875,4.6875)","[[1.2265625, 4.5625]]",[10.780338887125254],R
5,"(1.140625,4.625)","(0.296875,0.875)","(1.0546875,4.6875)","[[1.984375, 8.375]]",[2.960217535495758],R
6,"(1.51953125,6.53125)","(1.2265625,4.5625)","(1.984375,8.375)","[[2.69140625, 14.40625]]",[-12.831220891093835],E
7,"(2.337890625,11.390625)","(1.0546875,4.6875)","(2.69140625,14.40625)","[[7.470703125, 38.203125]]",[-26.48590685216186],E
8,"(5.0810546875,26.3046875)","(1.984375,8.375)","(7.470703125,38.203125)","[[4.306884765625, 21.822265625]]",[-22.009064559459205],C
